## I wanted to see more interesting airplanes, so I wrote some python and spun up a Linux box on AWS.

My inspiration for this project came from a [Pycon talk](https://www.youtube.com/watch?v=bK2iPDu7RDE&ab_channel=PyConUS) by Jessica Garson.  Jessica used python to send herself a text each day to figure out if she needed to move her car based on New York City parking regulations.

Since I love airplanes so much, I wanted to receive a notification around sunset with all of the interesting planes about to arrive into Phoenix Sky Harbor airport.  Since I live in a spot with a great view of the arrival flight pattern, I frequenly notice planes.  However, Phoenix doesn't have great diversity when it comes to airplanes.  The majority are Boeing 373s and Airbus A320s.  I've seen so many of these over the years, I would like my attention drawn to other interesting planes, such as an Airbus A330, or a Boeing 757, or a Boeing 747.

#### To pull this off I needed to gather several pieces of information:
1. run a once a day cron job to:
 - find the time of sunset for the current day and my current location
 - back up from sunset by 10 minutes (this is somewhat arbitrary, it just gives me a little more time to decide if any incoming planes are interesting, and coordinate some outdoor activity such as a walk to the grocery store)

1. at sunset minus 10 minutes fetch all incoming flights and exclude the most common aircraft models that arrive at the PHX airport.
 
1. send myself a text with interesting aircraft models, along with other interesting information such as the airport of origin

In [35]:
# import libraries
import requests
import pandas as pd
import datetime
from dateutil.parser import parse
import os
from twilio.rest import Client
from passwords import *

## Find Sunset for the current day

- this code gets run once a day at noon using a cron job
- it schedules an "at" job to run at sunset minus 10 minutes each day

In [2]:
# find today's date
current_date = str(datetime.datetime.today().date())

# assign variables for latitude and longitude
scottsdale_latitude='33.4942'
scottsdale_longitude='-111.9261'

# api call
sunset = requests.get(f'https://api.sunrise-sunset.org/json?lat={scottsdale_latitude}&lng={scottsdale_longitude}&formatted=0&date={current_date}')

# extract UTC time
sunset = sunset.json()['results']['sunset']

# back up from sunset by 10 minutes
sunset = parse(sunset) - pd.Timedelta(10, unit ='m')

# add time to string and schedule "at" job
sunset_job = f'echo python3 "flight_data.py" | at "{str(sunset)[11:16]}"'
sunrise_job = f'echo python3 "flight_data.py" | at "{str(sunrise)[11:16]}"'

# create "at" job
os.system(sunset_job)
os.system(sunrise_job)

In [3]:
str(sunset)

'2021-05-09 02:06:31+00:00'

## Get Flight Data
- this code is triggered using the above scheduled "at" job

In [4]:
# read in data from website
airport = 'KPHX'

df = pd.read_html(f'https://flightaware.com/live/airport/{airport}/enroute')[2]

# remove stacked column levels
df.columns = df.columns.droplevel()

# exclude uninteresting planes
common_planes = [
    'B73',
    'A32',
    'CRJ'
]

# filter out cases where no departure time or aircraft type identification is given
df = df[~df.Type.fillna('None').str.contains('|'.join(common_planes))].dropna(subset=['Departure'])

In [34]:
df

,Ident,Type,Origin,ScheduledDeparture Time,Departure,EstimatedArrival Time,Unnamed: 6_level_1,Unnamed: 7_level_1
2,AAL2059,A319,San Jose Int'l (KSJC),Sat 02:19PM PDT,Sat 02:14PM PDT,Sat 03:38PM MST,NaN,NaN
4,AAL504,A319,Salt Lake City Intl (KSLC),Sat 03:19PM MDT,Sat 03:28PM MDT,Sat 03:40PM MST,NaN,NaN
7,AAL2725,A21N,Reno/Tahoe Intl (KRNO),Sat 02:19PM PDT,Sat 02:14PM PDT,Sat 03:42PM MST,NaN,NaN
10,AAL702,A319,Tucson Intl (KTUS),Sat 03:23PM MST,Sat 03:19PM MST,Sat 03:45PM MST,NaN,NaN
15,AAL1441,A319,Spokane Intl (KGEG),Sat 01:19PM PDT,Sat 01:47PM PDT,Sat 03:48PM MST,NaN,NaN
16,FFT753,A20N,Denver Intl (KDEN),Sat 03:00PM MDT,Sat 03:21PM MDT,Sat 03:49PM MST,NaN,NaN


## Send myself a text
- using a Twilio free trial account I send flights in the above dataframe to myself as a text
- to keep the text messages more concise, I have limited the message to only include:
 - aircraft type
 - estimated arrival time
 - 4 digit origin airport code

In [32]:
# format flight data into text message
msg = '\n'.join(df['Type'] + ' - ' + 
                df['EstimatedArrival Time'].str.slice(start=4, stop=11) + ' - ' + 
                df.Origin.str.slice(start=-5, stop=-1))

# establish twilio credentials
client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

# send text message
client.api.account.messages.create(
    to=MY_PHONE_NUMBER,
    from_=TWILIO_PHONE_NUMBER,
    body=msg);

In [33]:
# example message string
msg

'A319 - 03:38PM - KSJC\nA319 - 03:40PM - KSLC\nA21N - 03:42PM - KRNO\nA319 - 03:45PM - KTUS\nA319 - 03:48PM - KGEG\nA20N - 03:49PM - KDEN'

## Next things to consider

1. run a cron job at regular intervals to aggregate flight data over time (maybe a sqlite db?)
1. mine historical data to determine which aircraft types are the most rare
1. send myself ad hoc texts based on especially rare arrivals
1. is there a way to find especially interesting planes based on tail number (Ident), such as Airforce 1? I could place them on a list and get an ad hoc text when one of these planes arrive